# Grafón: Qué electivas me conviene cursar?

Este grafo se encarga de analizar "gente que haya tenido experiencias facultativas similares", sin importar en que año o cuatrimestre fue. La idea es que si me fue parecido en varias materias a alguna otra persona, y esa persona curso alguna electiva que yo no, entonces esa materia es una buena candidata para mí.

Por ejemplo: Por más que Juan se haya recibido el año pasado, es una persona que se parece mucho a mí porque nos sacamos las mismas notas en muchas materias. Probablemente las electivas que Juan eligió, me sirvan a mí como guía.

## Cómo es el grafo?

El grafo analizado va a ser un grafo simple, y no un multigrafo: entre cada par de alumnos solo puede haber una arista. Manejamos la cardinalidad en el peso, en vez de tener múltiples aristas.

- Nodos: alumnos
- Aristas: conectar dos alumnos que hayan cursado misma materia y "les fue parecido".
    - A los dos nos fue muy bien (nos sacamos entre 8 y 10)    
    - A los dos nos fue más o menos (nos sacamos un 6 o un 7)
    - A los dos nos fue mal (nos sacamos un 4 o un 5)
- Peso: La inversa* de la cantidad de materias en donde somos similares

*Calculamos la inversa porque mientras mas similares son, menor peso queremos que haya entre la arista, para que más cercanos esten

### Dos formas de armar el análisis
// ToDo: que hacemos aca? Elegimos uno, o presentamos ambos en el mismo notebook?
- sólo correr sobre materias electivas
- correr sobre todas las materias, pero filtrar el output por electivas

In [ ]:
import pandas as pd

df = pd.read_pickle('fiuba-map-data.pickle')
df.sample(3)

In [ ]:
categories = {
    4: 0,
    5: 0,
    6: 1,
    7: 1,
    8: 2,
    9: 2,
    10: 2
}

df_rel = df[['Padron', 'materia_id', 'materia_nota']].copy()

# Sacamos materias en final y a cursar
df_rel = df_rel[df_rel['materia_nota'] >= 4]

# Sacamos gente que no le pone la nota a su fiubamap y deja que se saco (casi) todos 4s directamente
df_rel['mediana'] = df_rel.groupby('Padron')['materia_nota'].transform('median')
df_rel = df_rel[df_rel['mediana'] > 5]

df_rel['nota_categoria'] = df_rel['materia_nota'].apply(lambda x: categories[x])

# Juntamos el grafo con si mismo para tener la similiritud entre cada par de padrones
df_simil = pd.merge(df_rel, df_rel, on=['materia_id', 'nota_categoria'])
df_simil = df_simil[df_simil['Padron_x'] != df_simil['Padron_y']]
df_simil = df_simil.reset_index()

df_simil = df_simil[['materia_id', 'nota_categoria', 'Padron_x', 'materia_nota_x', 'Padron_y', 'materia_nota_y']]
display(df_simil.sample(3))
df_simil.shape

Esto resultaría en un grafo con pocos nodos y millones de aristas. Por eso se busca un enfoque de unificar aristas mediante algún tipo de criterio y de esta forma tener sólo una arista entre padrones.

In [ ]:
import numpy as np

df_simil_agg = df_simil.groupby(['Padron_x', 'Padron_y']).agg(cant_materias_similares=('materia_id', 'count'))
df_simil_agg = df_simil_agg.reset_index()

df_simil_agg['Padron_min'] = df_simil_agg[['Padron_x', 'Padron_y']].min(axis=1)
df_simil_agg['Padron_max'] = df_simil_agg[['Padron_x', 'Padron_y']].max(axis=1)
df_simil_agg = df_simil_agg.drop_duplicates(['Padron_min', 'Padron_max']).reset_index()

# ToDo: fijarse si elevar a un K acá
df_simil_agg['inv_cant_materias_similares'] = df_simil_agg['cant_materias_similares'].max() - df_simil_agg['cant_materias_similares'] + 1
df_simil_agg = df_simil_agg[['Padron_x', 'Padron_y', 'cant_materias_similares', 'inv_cant_materias_similares']]

display(df_simil_agg.sample(3))
df_simil_agg.shape

In [ ]:
df_simil_agg.sort_values('cant_materias_similares', ascending=False).head(15)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.from_pandas_edgelist(df_simil_agg, 
                            source='Padron_x', 
                            target='Padron_y', 
                            edge_attr='inv_cant_materias_similares',
                            create_using=nx.Graph())

print(G)
print(f"""
  El diámetro de la red: {nx.diameter(G)}
  El grado promedio de la red: {sum([n[1] for n in G.degree()]) / len(G):.2f}
  Puentes globales: {list(nx.bridges(G))}
""")

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(30,10))
ax = axes.flatten()

# ToDo: Elegir un randomsample (si? no?) para graficar y que se entienda mejor el grafico. Tal vez el grupo del tp puede ser el sample
nx.draw_networkx(G, pos=nx.circular_layout(G), width=0.0005, node_size=50, with_labels=False, ax=ax[0])
nx.draw_networkx(G, pos=nx.spiral_layout(G),   width=0.005,  node_size=50, with_labels=False, ax=ax[1])

## Comunidades

In [ ]:
from networkx.algorithms import community
import matplotlib.colors as mcolors
import random

random.seed(42)
plt.figure(figsize=(30,10))

louvain = community.louvain_communities(G, weight='inv_cant_materias_similares')
draw_nodes = {}
colors = random.sample(list(mcolors.CSS4_COLORS), len(louvain))
for louvaincommunity, color in zip(louvain, colors):
    draw_nodes.update({n: color for n in louvaincommunity})
    
plt.title(f"{len(louvain)} Louvain Communities")
nx.draw_networkx(G, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 width=0.0005, 
                 pos=nx.random_layout(G, seed=42),
                 font_size=10)

In [ ]:
from setup import PADRON, CARRERA, plan_estudios

def nestedsearch(el, lst_of_sets):
    return list(filter(lambda lst: el in lst, lst_of_sets))[0]

def materias_padron(padron):
    return df[(df['Padron'] == padron) & (df['materia_nota'] >= 4)]['materia_id'].values

def sugerir_electivas(padron):
    min_alumnos, max_alumnos = 6, 20
    max_iteraciones = 25

    i = 0
    grupo = []
    for i in range(max_iteraciones):
        louvain = community.louvain_communities(G, weight='inv_cant_materias_similares', resolution=1+(i*0.01))
        comunidad = nestedsearch(padron, louvain)
        if min_alumnos <= len(comunidad) <= max_alumnos:
            grupo = comunidad
            break
        elif not grupo or (len(comunidad) >= max_alumnos and (len(comunidad) - max_alumnos <= len(grupo) - max_alumnos)):
            grupo = comunidad
        i+=1
    
    df_sugerencias = df_rel[df_rel['Padron'].isin(grupo)].groupby('materia_id').agg(cant_alumnos_similares=('materia_id', 'count'))
    df_sugerencias = df_sugerencias[~df_sugerencias.index.isin(materias_padron(padron))]
    
    df_materias = pd.read_json(plan_estudios(CARRERA))
    df_sugerencias = pd.merge(df_sugerencias, df_materias, left_on='materia_id', right_on="id")
    df_sugerencias = df_sugerencias[df_sugerencias['categoria'] == 'Materias Electivas']
    df_sugerencias = df_sugerencias[['id', 'materia', 'creditos', 'cant_alumnos_similares']].sort_values('cant_alumnos_similares', ascending=False)
    return df_sugerencias.reset_index(drop=True)

electivas = sugerir_electivas(PADRON)
electivas.head(10)